In [65]:
from pymongo import MongoClient
import pandas as pd
import datetime

# Open Database and find history data collection
client = MongoClient()
db = client.test_database
shdaily = db.indexdata

# KDJ calculation formula
def KDJCalculation(K1, D1, high, low, close):
    # input last K1, D1, max value, min value and current close value
    #设定KDJ基期值
    #count = 9
    #设定k、d平滑因子a、b，不过目前已经约定俗成，固定为1/3
    a = 1.0/3
    b = 1.0/3
    # 取得过去count天的最低价格
    low_price = low #low.min() #min(list1)
    # 取得过去count天的最高价格
    high_price = high #high.max() #max(list1)
    # 取得当日收盘价格
    current_close = close
    if high_price!=low_price:
        #计算未成熟随机值RSV(n)＝（Ct－Ln）/（Hn-Ln）×100
        RSV = (current_close-low_price)/(high_price-low_price)*100
    else:
        RSV = 50
    #当日K值=(1-a)×前一日K值+a×当日RSV
    K2=(1-a)*K1+a*RSV
    #当日D值=(1-a)×前一日D值+a×当日K值
    D2=(1-b)*D1+b*K2
    #计算J值
    J2 = 3*K2-2*D2
    #log.info("Daily K1: %s, D1: %s, K2: %s, D2: %s, J2: %s" % (K1,D1,K2,D2,J2))
    return K1,D1,K2,D2,J2



# Put the first dataset in



# List the data 
# initial Values
K1 = 50
D1 = 50

# for each day, calculate data and insert into db
for d in shdaily.find()[:10]:
    date = d['date']
    datalist = pd.DataFrame(list(shdaily.find({'date':{"$lte": date}}).sort('date', -1)))
    data = datalist[:9]
    
    # get previous KDJ data from database
    K1 = data.ix[1]['KDJ_K']
    D1 = data.ix[1]['KDJ_D']
    
    high = data['high'].values
    low  = data['low'].values
    close = data[:1]['close'].values
    K1,D1,K2,D2,J2 = KDJCalculation(K1,D1,max(high),min(low),close)
    d['KDJ_K'] = K2[0]
    d['KDJ_D'] = D2[0]
    d['KDJ_J'] = J2[0]
    
#     K1 = K2
#     D1 = D2
    print d


#datalist = pd.DataFrame(list(shdaily.find().sort('date', -1)))

#date1 = datetime.strptime("01/01/16", "%d/%m/%y")


# List out the data before or equal a specific date
#list(shdaily.find({'date':{"$lte":'2016-02-08'}}).sort('date', -1))


# Get last day KDJ data from database
datalist = pd.DataFrame(list(shdaily.find({'date':{"$lte": '2016-02-10'}}).sort('date', -1)))
data = datalist.ix[1]
data['KDJ_K']


# Save data to db 


# data = datalist[:9]

# data



# K1 = 50
# D1 = 50
# high = data['high'].values
# low  = data['low'].values
# close = data[:1]['close'].values

# K1,D1,K2,D2,J2 = KDJCalculation(K1,D1,max(high),min(low),close)



_id             57080f973d10a288385cf4ca
close                            2781.02
date                          2016-02-04
high                              2793.3
low                              2751.31
ma10                             2769.24
ma20                             2877.77
ma5                              2739.26
open                             2751.43
p_change                            1.52
price_change                       41.77
v_ma10                       1.73044e+06
v_ma20                       1.94013e+06
v_ma5                        1.63983e+06
volume                       1.70382e+06
Name: 1, dtype: object

In [1]:
# Another KDJ Calculation based on dataframe
def CalculateKDJ(stock_data):
    # Initiate KDJ parameters
    endday = pd.datetime.today()
    N1= 9
    N2= 3
    N3= 3
    # Perform calculation
    #stock_data = get_price(stock, end_date=endday)
    low_list = pd.rolling_min(stock_data['LowPx'], N1)
    low_list.fillna(value=pd.expanding_min(stock_data['LowPx']), inplace=True)
    high_list = pd.rolling_max(stock_data['HighPx'], N1)
    high_list.fillna(value=pd.expanding_max(stock_data['HighPx']), inplace=True)
    #rsv = (stock_data['ClosingPx'] - low_list) / (high_list - low_list) * 100
    
    rsv = (stock_data['ClosingPx'] - stock_data['LowPx']) / (stock_data['HighPx'] - stock_data['LowPx']) * 100
    stock_data['KDJ_K'] = pd.ewma(rsv, com = N2)
    stock_data['KDJ_D'] = pd.ewma(stock_data['KDJ_K'], com = N3)
    stock_data['KDJ_J'] = 3 * stock_data['KDJ_K'] - 2 * stock_data['KDJ_D']
    KDJ = stock_data[['KDJ_K','KDJ_D','KDJ_J']]
    return KDJ